In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 

In [69]:
code='NIFTY_BANK.csv'
df=pd.read_csv(code)
df.rename(columns={'Date ':'Date','Open ':'Open','High ':'High','Low ':'Low','Close ':'Close'},inplace=True)
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close'], dtype='object')

In [70]:
df.set_index('Date',inplace=True)
df.index=pd.to_datetime(df.index,format="%d/%m/%y")

In [71]:
df.tail()

,Open,High,Low,Close
Date,,,,
2024-02-27,46480.20,46722.25,46324.90,46588.05
2024-02-28,46640.90,46754.55,45852.55,45963.15
2024-02-29,45881.45,46329.65,45661.75,46120.90
2024-03-01,46218.00,47342.25,46218.00,47286.90
2024-03-02,47377.45,47433.85,47237.00,47297.50


In [145]:
class Backtest:
    
    window=1
    entry_price,exit_price,entry_date,exit_date=0,0,0,0
    trade_direction=str()
    
    def __init__(self, df, max_sl,bep_tr):
        self.df=df
        self.max_sl=max_sl
        self.bep_tr=bep_tr
        self.trades=[]
        self.trade_dt=0
        self.initial_investment=100000
        self.max_dd=0
        self.calmer=0
        self.avg_pnl=0
        self.Pivot()
        self.Prev_Pivot_idx()
        self.Prev_Pivot()
        self.execution()
        self.trade_data()
        self.trade_metrics()
        
    def Pivot(self):
        
        pivot=[0 for i in range(len(self.df))]
        
        for i in range(len(self.df)):
            
            if (i - self.window) < 0 or (i+self.window>=len(self.df)):
                pivot[i]=0
            elif self.df.Close.iloc[i] > self.df.Close[i-self.window] and self.df.Close.iloc[i] > self.df.Close[i+self.window] and self.df.Close.iloc[i] < self.df.Close[i-self.window] and self.df.Close.iloc[i] < self.df.Close[i+self.window]:
                pivot[i]=3
            elif self.df.Close.iloc[i] > self.df.Close[i-self.window] and self.df.Close.iloc[i] > self.df.Close[i+self.window]:
                pivot[i]=1
            elif self.df.Close.iloc[i] < self.df.Close[i-self.window] and self.df.Close.iloc[i] < self.df.Close[i+self.window]:
                pivot[i]=2
        
        self.df['isPivot']=pivot 
        
    def Prev_Pivot_idx(self):
        
        self.df = df.reset_index()
        
    
        self.df = self.df.rename_axis('dummy_idx').reset_index()
        self.df['prev_idx'] = self.df.groupby('isPivot')['dummy_idx'].shift()
    
        # convert floats to integers (and NaN to <NA>)
        self.df['prev_idx'] = self.df['prev_idx'].astype('Int64')
        self.df['prev_idx']=self.df['prev_idx'].fillna(0)
        
        self.df.drop(['dummy_idx'],axis=1)
        self.df.set_index('Date',inplace=True)

    def Prev_Pivot(self):
        
        prev_pivot=[0 for i in range(len(self.df))]
        cnt_pvt=0
        
        for i in range(len(self.df)):
            
            if (self.df.isPivot.iloc[i] == 1 or self.df.isPivot.iloc[i] == 2) and cnt_pvt < 2:
                cnt_pvt=cnt_pvt+1
            elif self.df.isPivot.iloc[i] == 1 or self.df.isPivot.iloc[i] == 2:
                idx=self.df.prev_idx.iloc[i]
                prev_pivot[i]=self.df.Close.iloc[idx]        
        
        self.df['Prev_Pivot']=prev_pivot
    
    def trade_log(self):
        
        self.trades.append({'Trade Direction':self.trade_direction,
                        'Entry Date':self.entry_date,
                       'Entry Price':self.entry_price,
                       'Exit Date': self.exit_date,
                      'Exit Price': self.exit_price,
                      'Profit': self.exit_price - self.entry_price if self.trade_direction=='Long' else self.entry_price - self.exit_price})  
       
    def trade_data(self):
        
        self.trade_dt=pd.DataFrame(self.trades)
    
    def trade_metrics(self):
        
        mean_pnl=0
        
        self.trade_dt['Cum_Pnl']=self.trade_dt['Profit'].cumsum()
        self.trade_dt['Equity']=self.trade_dt['Cum_Pnl']+self.initial_investment
        self.trade_dt['Peak']=self.trade_dt['Equity'].cummax()
        self.trade_dt['Drawdown']=((self.trade_dt['Equity']-self.trade_dt['Peak'])/self.trade_dt['Peak'])*100
        #self.trade_dt['Drawdown']=self.trade_dt['Equity']-self.trade_dt['Peak']
        self.max_dd=self.trade_dt['Drawdown'].min()
        mean_pnl=self.trade_dt.Cum_Pnl.iloc[-1]/2
        self.avg_pnl=(mean_pnl/self.initial_investment)*100
        self.calmer=abs(self.avg_pnl/self.max_dd)
        
    @staticmethod
    def long_stop(long_price,system_low,sl):
        system_points=long_price - system_low
    
        if  long_price*(sl/100) < system_points:
            stop_price=round(long_price*(1-(sl/100)),2)
        else:
            stop_price=system_low
    
        return stop_price
    
    @staticmethod
    def short_stop(short_price,system_high,sl):
        system_points=system_high - short_price
    
        if short_price*(sl/100) < system_points:
            stop_price=round(short_price*(1+(sl/100)),2)
        else:
            stop_price=system_high 
    
        return stop_price
    
    def execution(self):
        
        swing_high,swing_low=0,0
        long_price,short_price=0,0
        st,gap_up,gap_down=0,0,0
        pos_sl,bep=0,0
        
        for i in range(len(self.df)):
            
            if self.df.isPivot.iloc[i]==1:
                if self.df.Prev_Pivot.iloc[i]==0 or self.df.Prev_Pivot.iloc[i] > self.df.Close.iloc[i] or st==1:
                    swing_high=self.df.Close.iloc[i]
                else:
                    swing_high=self.df.Prev_Pivot.iloc[i]
                gap_up=0
            elif self.df.isPivot.iloc[i]==2:
                if self.df.Prev_Pivot.iloc[i]==0 or self.df.Prev_Pivot.iloc[i] < self.df.Close.iloc[i] or st==2:
                    swing_low=self.df.Close.iloc[i]
                else:
                    swing_low=self.df.Prev_Pivot.iloc[i]
                gap_down=0
            
            if st==1:
                if self.df.Open.iloc[i] < swing_low*(1-(self.max_sl/100)):
                    gap_down=1
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Open.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif self.df.Open.iloc[i] < self.df.Close.iloc[i-1]*(1-(self.max_sl/100)):
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif pos_sl!=0 and self.df.Close.iloc[i] < pos_sl:
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif self.df.Close.iloc[i] > long_price*(1+(self.bep_tr/100)) and bep==0:
                    pos_sl=self.entry_price
                    bep=bep+1
                
            if st==2:
                if self.df.Open.iloc[i] > swing_high*(1+(self.max_sl/100)):
                    gap_up=1
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Open.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif self.df.Open.iloc[i] > self.df.Close.iloc[i-1]*(1+(self.max_sl/100)):
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif pos_sl!=0 and self.df.Close.iloc[i] > pos_sl:
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif self.df.Close.iloc[i] < short_price*(1-(self.bep_tr/100)) and bep==0:
                    pos_sl=self.entry_price
                    bep=bep+1
                
            
            if swing_high!=0 and self.df.Close.iloc[i] > swing_high and gap_up==0 and (st==0 or st==2):
                long_price=swing_high
                pos_sl,bep=0,0
                if st==2:
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    self.entry_date=self.df.index[i]
                    self.entry_price=self.df.Close.iloc[i]
                    self.trade_direction='Long'
                else:
                    self.entry_date=self.df.index[i]
                    self.entry_price=self.df.Close.iloc[i]
                    self.trade_direction='Long'
                st=1
                if Backtest.long_stop(long_price,swing_low,self.max_sl) != swing_low:
                    pos_sl=Backtest.long_stop(long_price,swing_low,self.max_sl)
                    #print(pos_sl)
                    #print(self.df.index[i])
                else:
                    #print(pos_sl)
                    #print(self.df.index[i])
                    pos_sl=0
                
            elif swing_low!=0 and self.df.Close.iloc[i] < swing_low and gap_down==0 and (st==0 or st==1):
                short_price=swing_low
                pos_sl,bep=0,0
                if st==1:
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    self.entry_date=self.df.index[i]
                    self.entry_price=self.df.Close.iloc[i]
                    self.trade_direction='Short'
                else:
                    self.entry_date=self.df.index[i]
                    self.entry_price=self.df.Close.iloc[i]
                    self.trade_direction='Short'
                st=2
                if Backtest.short_stop(short_price,swing_high,self.max_sl) != swing_high:
                    pos_sl=Backtest.short_stop(short_price,swing_high,self.max_sl)
                    print(pos_sl)
                    print(self.df.index[i])
                    print(short_price)
                else:
                    pos_sl=0
                
             

In [146]:
instance=Backtest(df,2,2)

38240.06
2022-01-24 00:00:00
37490.25
38755.36
2022-02-14 00:00:00
37995.45
41197.6
2023-03-13 00:00:00
40389.8


In [147]:
instance.df.head(20)

,dummy_idx,Open,High,Low,Close,isPivot,prev_idx,Prev_Pivot
Date,,,,,,,,
2022-01-03,0,35585.20,36492.10,35526.60,36421.90,0,0,0.00
2022-01-04,1,36551.25,36887.80,36374.40,36840.15,0,0,0.00
2022-01-05,2,36943.55,37862.40,36756.35,37695.90,1,0,0.00
2022-01-06,3,37242.55,37752.50,37058.45,37490.25,2,0,0.00
2022-01-07,4,37667.05,38134.85,37427.80,37739.60,0,1,0.00
2022-01-10,5,37930.55,38400.35,37929.35,38347.90,0,4,0.00
2022-01-11,6,38370.00,38504.60,38031.75,38442.20,0,5,0.00
2022-01-12,7,38719.60,38851.45,38604.60,38727.55,1,2,37695.90
2022-01-13,8,38717.55,38717.55,38376.15,38469.95,0,6,0.00


In [148]:
instance.trade_dt

,Trade Direction,Entry Date,Entry Price,Exit Date,Exit Price,Profit,Cum_Pnl,Equity,Peak,Drawdown
0,Long,2022-01-07,37739.60,2022-01-21,37574.30,-165.30,-165.30,99834.70,99834.70,0.000000
1,Short,2022-01-24,36947.55,2022-02-01,38505.50,-1557.95,-1723.25,98276.75,99834.70,-1.560530
2,Long,2022-02-01,38505.50,2022-02-07,37995.45,-510.05,-2233.30,97766.70,99834.70,-2.071424
3,Short,2022-02-14,36908.55,2022-03-10,34475.60,2432.95,199.65,100199.65,100199.65,0.000000
4,Long,2022-03-16,35748.25,2022-03-24,35527.10,-221.15,-21.50,99978.50,100199.65,-0.220709
...,...,...,...,...,...,...,...,...,...,...
59,Short,2024-01-08,47450.25,2024-01-29,45442.35,2007.90,17070.75,117070.75,117070.75,0.000000
60,Long,2024-01-29,45442.35,2024-02-08,45012.00,-430.35,16640.40,116640.40,117070.75,-0.367598
61,Short,2024-02-08,45012.00,2024-02-14,45908.30,-896.30,15744.10,115744.10,117070.75,-1.133204
62,Long,2024-02-14,45908.30,2024-02-28,45963.15,54.85,15798.95,115798.95,117070.75,-1.086352


In [149]:
instance.max_dd

-2.3017608874360365

In [150]:
instance.avg_pnl

7.237599999999998

In [151]:
instance.calmer

3.144375264826948